## Homework6
時系列データを含むデータベースファイル (data.sqlite3) に含まれるデータについて、goods_genre_id ごとに price の平均値を求めてください。時系列データには goods_genre_id の情報は含まれていないので、複数のテーブルを組み合わせて処理する必要があります。  
データの詳細は以下を参照してください。

### データ一覧
今回与えられるデータは data.sqlite3 という sqlite3 形式のデータです。
以下の 2 つのテーブルが含まれています。
* data ... 時系列データ
* goods ... 商品に関する補足情報

### カラム情報一覧
データに含まれるカラムの情報は以下です。
* index ... (店舗, 商品) の各組み合わせに対して割り振られるID
* store_id ... 店舗ID
* goods_id ... 商品ID
* goods_genre_id ... 商品カテゴリID
* units_sold_day ... 商品の一日の売り上げ個数
* units_sold_month ... 商品のひと月の売り上げ個数
* price ... 商品の現在の値段
* yy-mm-dd ... 日付 (yy/mm/dd形式)
* num_month ... 便宜上各月にわりふられた値 (平成25年1月=1, 平成25年2月=2,...,平成27年9月=33)
* goods_name ... 商品名
* store_name ... 店舗名
* goods_genre_name ... 商品カテゴリ名

### 注意事項
- データ処理はSQLiteでの操作を推奨します。補足資料「SQLite_manual.pdf」「SQLite_manual.ipynb」を参照ください。
- SQLiteのインストール/ダウンロードが上手くいかない場合、python上での操作も可能です。補足資料「csv_export_python.pdf」「csv_export_python.ipynb」を参照ください。
- 一部の処理に時間がかかる場合があります。

### 提出/採点
回答は**Omnicampus**上で提出してください。提出するデータの形式は以下のような**csvファイル**となります。
 
goods_genre_id|avg_price
---|---
1|25000
3|26500
4|25000
…|…
196|20000
200|27000


提出ファイルがこの形式に準じていなかった場合にはスコアが1000となるので、その際はファイル形式を確認しましょう。  

In [1]:
import sqlite3
import csv

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 学習データ
dbpath = '/content/drive/MyDrive/プログラミング、AI/2022年前期東大講座/GCI/6.第七回/data.sqlite3'

In [4]:
conn = sqlite3.connect(dbpath)

In [5]:
cur = conn.cursor()

In [6]:
# テーブル名の確認
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('data',)
('goods',)


In [32]:
select * from  data
union
select * from  goods;

SyntaxError: ignored

In [33]:
select * from goods;

SyntaxError: ignored

In [19]:
# カラム名(CREATE TABLE文)の確認
cur.execute('select sql from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)',)
('CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)',)


In [20]:
# 全情報の確認 (type, name, tbl_name, rootpage, sql)
cur.execute('select * from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('table', 'data', 'data', 2, 'CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)')
('table', 'goods', 'goods', 12, 'CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)')


In [21]:
cur.execute('select * from goods')
print(cur.fetchone()) #1レコード目の取得
print(cur.fetchone()) #2レコード目の取得

('! POWER IN glamor (PLAST.) D', '1', '41')
('! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]', '2', '77')


In [22]:
'''
cur.execute('select * from data')
for row in cur.fetchall():
  print(row)
'''

"\ncur.execute('select * from data')\nfor row in cur.fetchall():\n  print(row)\n"

In [23]:
#テーブルから特定の列を取得
cur.execute('select store_id from data')
with open('out.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [25]:
'''
select * from  goods;
'''

'\nselect * from  goods;\n'

In [26]:
#テーブルから全ての列を取得
cur.execute('select * from data')
with open('out.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [27]:
# 処理を確定
conn.commit()

In [28]:
cur.close()

In [29]:
conn.close()

In [30]:
import pandas as pd

In [31]:
pd.read_csv('out.csv', sep=',')

,yy_mm_dd,num_month,store_id,goods_id,price,units_sold_day
0,25-01-29,-33,43,17718,44943,2
1,25-01-08,-33,32,8648,14949,1
2,25-01-16,-33,53,10168,30158,1
3,25-01-23,-33,31,1005,9922,1
4,25-01-12,-33,27,19861,14988,0
...,...,...,...,...,...,...
995,25-01-06,-33,2,19739,7568,1
996,25-01-07,-33,55,19233,40297,3
997,25-01-19,-33,24,7771,108849,1
998,25-01-11,-33,55,3321,257884,1


In [21]:
import numpy as np
import pandas as pd
import torch


# 学習データ
dbpath = np.load('/content/drive/MyDrive/プログラミング、AI/2022年前期東大講座/GCI/6.第七回/data.sqlite3')

dbpath.head()

ValueError: ignored